In [2]:
from ortools.linear_solver import pywraplp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.offsets import DateOffset

# Functions that produce a value of satisfaction of stakeholders

In [3]:
def line_solver(x_1, y_1, x_2, y_2):
    m = (y_2 - y_1) / (x_2 - x_1)
    c = y_1 - m * x_1
    return m, c
    

In [4]:
august_prev = line_solver(73.76, 21.75, 74.68, 0)
sept_prev = line_solver(73.76, 15.8,74.68,0)
may_prev = line_solver(73.76, 8.75,74.68,0)
oct_prev = line_solver(73.76, 7.5,74.68,0)
april_prev = line_solver(73.76, 3.75,74.68,0)
nov_prev = line_solver(73.76, 2.5,74.68,0)
june_prev = may_prev
july_prev = august_prev

august_after = line_solver(75.35, 0, 75.9, 6.7)
sept_after = line_solver(75.35, 0,75.9, 5)
oct_after = line_solver(75.35, 0,75.9, 2.5)
may_after = line_solver(75.35, 0,75.9, 2.5)
april_after = line_solver(75.35, 0,75.9, 1)
nov_after = line_solver(75.35, 0,75.9, 1)
june_after = may_after
july_after = august_after

In [5]:
print(august_prev, august_after)

(-23.641304347826043, 1765.532608695649) (12.181818181817931, -917.899999999981)


In [148]:
def house_owner1(wl, months):
  if months == 8:
    return (74.68 - wl) * august_prev[0]
  if months == 9:
    return (74.68 - wl) * sept_prev[0]
  if months == 10:
    return (74.68 - wl) * oct_prev[0]
  if months == 5:
    return (74.68 - wl) * may_prev[0]
  if months == 4:
    return (74.68 - wl) * april_prev[0]
  if months == 11:
    return (74.68 - wl) * nov_prev[0]
  if months == 6:
    return (74.68 - wl) * june_prev[0]
  if months == 7:
    return (74.68 - wl) * july_prev[0]
  
def house_owner2(wl, months):
  if months == 8: 
    return (75.35 - wl) * august_after[0]
  if months == 9:
    return (75.35 - wl) * sept_after[0]
  if months == 10:
    return (75.35 - wl) * oct_after[0]
  if months == 5:
    return (75.35 - wl) * may_after[0]
  if months == 4:
    return (75.35 - wl) * april_after[0]
  if months == 11:
    return (75.35 - wl) * nov_after[0]
  if months == 6:
    return (75.35 - wl) * june_after[0]
  if months == 7:
    return (75.35 - wl) * july_after[0]


In [ ]:
# replace the original linear funciton 


In [107]:
# different months can have different factors
def boating_1(wl):
  return (75 - wl) * 10
def boating_2(wl):
  return (wl - 74) * 10

In [155]:
power_cap = (912.0 + 1045.0) * 10.0
Q_arr = []
def hydropower(wl, month):
  Q = 0.104 # Can be predicted by month
  g = 9.8
  return 0.9 * Q * g * wl

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [108]:
def navigation(wl):

  return (wl - 74.8) * 10

In [49]:
def envir(wl, month):
  return 0

# Calculating the optimal water level

In [157]:
months = [i + 1 for i in range(12)]

In [151]:
# Constants in the constraints can be replaced by functions in the previous section

def Optwl(months):
    """Linear programming sample."""
    # Instantiate a Glop solver, naming it LinearExample.
    solver = pywraplp.Solver.CreateSolver("GLOP")
    if not solver:
        return

    # Water level
    wl = solver.NumVar(0, solver.infinity(), "water level")
    # y = solver.NumVar(0, solver.infinity(), "y")

    print("Number of variables =", solver.NumVariables())

    # Constraints of water level:
    solver.Add(wl >= 73.0)
    solver.Add(wl <= 77.0)

    # Constraints of house owners:
    solver.Add(house_owner1(wl) >= -1.0)
    solver.Add(house_owner2(wl) >= -10)

    # Constraints of boaters:
    solver.Add(boating_1(wl) >= -5)
    solver.Add(boating_2(wl) >= -5)

    # Constraints of hydropower stations:
    solver.Add(hydropower(wl, 1) <= power_cap)
    solver.Add(hydropower(wl, 1) >= 30)

    # Constraints of shipping:
    solver.Add(navigation(wl) >= -7)

    print("Number of constraints =", solver.NumConstraints())

    # Objective function:
    solver.Maximize(0.3 * house_owner1(wl) + 0.1 * house_owner2(wl) + 0.3 * hydropower(wl, 1) + 0.2 * navigation(wl) + 0.1 * (boating_1(wl) + boating_2(wl)))

    # Solve the system.
    print(f"Solving with {solver.SolverVersion()}")
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("Solution:")
        print(f"Objective value = {solver.Objective().Value()}")
        print(f"Water level = {wl.solution_value()}m")
        #print(f"y = {y.solution_value()}")
    else:
        print("The problem does not have an optimal solution.")

    print("\nAdvanced usage:")
    print(f"Problem solved in {solver.wall_time():d} milliseconds")
    print(f"Problem solved in {solver.iterations():d} iterations")

In [152]:
Optwl(months)

Number of variables = 1
Number of constraints = 8
Solving with Glop solver v9.8.3296
Solution:
Objective value = 24.611134400000022
Water level = 74.1m

Advanced usage:
Problem solved in 5 milliseconds
Problem solved in 0 iterations
